In [1]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
print(boto3.__version__)

1.40.16


In [2]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

[2025-08-28 14:20:55,214] p16253 {credentials.py:1136} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [4]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-east-1', '637423659006')

In [47]:
inference_profile = "us.mistral.mistral-large-2402-v1:0" 
foundation_model = inference_profile[3:]
foundation_model

'mistral.mistral-large-2402-v1:0'

In [48]:
suffix = f"{region}-{account_id}"
agent_name = "hr-assistant-function-def"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_description = "Agent for providing HR assistance to manage vacation time"
agent_instruction = "You are an HR agent, helping employees understand HR policies and manage vacation time"
agent_action_group_name = "VacationsActionGroup"
agent_action_group_description = "Actions for getting the number of available vacations days for an employee and confirm new time off"
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'
lambda_function_name = f'{agent_name}-{suffix}'

In [7]:
lambda_function_name

'hr-assistant-function-def-us-east-1-637423659006'

In [8]:
# creating employee database to be used by lambda function
import sqlite3
import random
from datetime import date, timedelta

# Connect to the SQLite database (creates a new one if it doesn't exist)
conn = sqlite3.connect('employee_database.db')
c = conn.cursor()

# Create the employees table
c.execute('''CREATE TABLE IF NOT EXISTS employees
                (employee_id INTEGER PRIMARY KEY AUTOINCREMENT, employee_name TEXT, employee_job_title TEXT, employee_start_date TEXT, employee_employment_status TEXT)''')

# Create the vacations table
c.execute('''CREATE TABLE IF NOT EXISTS vacations
                (employee_id INTEGER, year INTEGER, employee_total_vacation_days INTEGER, employee_vacation_days_taken INTEGER, employee_vacation_days_available INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Create the planned_vacations table
c.execute('''CREATE TABLE IF NOT EXISTS planned_vacations
                (employee_id INTEGER, vacation_start_date TEXT, vacation_end_date TEXT, vacation_days_taken INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Generate some random data for 10 employees
employee_names = ['John Doe', 'Jane Smith', 'Bob Johnson', 'Alice Williams', 'Tom Brown', 'Emily Davis', 'Michael Wilson', 'Sarah Taylor', 'David Anderson', 'Jessica Thompson']
job_titles = ['Manager', 'Developer', 'Designer', 'Analyst', 'Accountant', 'Sales Representative']
employment_statuses = ['Active', 'Inactive']

for i in range(10):
    name = employee_names[i]
    job_title = random.choice(job_titles)
    start_date = date(2015 + random.randint(0, 7), random.randint(1, 12), random.randint(1, 28)).strftime('%Y−%m−%d')
    employment_status = random.choice(employment_statuses)
    c.execute("INSERT INTO employees (employee_name, employee_job_title, employee_start_date, employee_employment_status) VALUES (?, ?, ?, ?)", (name, job_title, start_date, employment_status))
    employee_id = c.lastrowid

for year in range(date.today().year, date.today().year - 3, -1):
    total_vacation_days = random.randint(10, 30)
    days_taken = random.randint(0, total_vacation_days)
    days_available = total_vacation_days - days_taken

    c.execute(
        """
        INSERT INTO vacations (
            employee_id, year, employee_total_vacation_days, 
            employee_vacation_days_taken, employee_vacation_days_available
        ) VALUES (?, ?, ?, ?, ?)
        """,
        (employee_id, year, total_vacation_days, days_taken, days_available)
    )

    # Generate some planned vacations for the current employee and year
    num_planned_vacations = random.randint(0, 3)
    for _ in range(num_planned_vacations):
        # pick a random start date
        start_date = date(year, random.randint(1, 12), random.randint(1, 28))
        # add between 1–14 days
        end_date = start_date + timedelta(days=random.randint(1, 14))
        # calculate days taken
        days_taken = (end_date - start_date).days

        # insert into DB using ISO format (YYYY-MM-DD)
        c.execute(
            """
            INSERT INTO planned_vacations (
                employee_id, vacation_start_date, vacation_end_date, vacation_days_taken
            ) VALUES (?, ?, ?, ?)
            """,
            (employee_id, start_date.isoformat(), end_date.isoformat(), days_taken)
        )

# Commit the changes and close the connection
conn.commit()
conn.close()

In [9]:
%%writefile lambda_function.py
import os
import json
import shutil
import sqlite3
from datetime import datetime


def get_available_vacations_days(employee_id):
    conn = sqlite3.connect('/tmp/employee_database.db')
    c = conn.cursor()

    if employee_id:
        c.execute("""
            SELECT employee_vacation_days_available
            FROM vacations
            WHERE employee_id = ?
            ORDER BY year DESC
            LIMIT 1
        """, (employee_id,))

        available_vacation_days = c.fetchone()

        if available_vacation_days:
            available_vacation_days = available_vacation_days[0]
            conn.close()
            return available_vacation_days
        else:
            conn.close()
            return f"No vacation data found for employed_id {employee_id}"
    else:
        conn.close()
        raise Exception("No employee id provided")

Writing lambda_function.py


In [49]:

try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

{'ResponseMetadata': {'RequestId': 'b221fded-1740-410c-9773-9afde5ee342b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 28 Aug 2025 14:49:40 GMT',
   'x-amzn-requestid': 'b221fded-1740-410c-9773-9afde5ee342b',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [50]:

# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda_function.py")
z.write("employee_database.db")
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

In [51]:
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:*::foundation-model/{foundation_model}",
                f"arn:aws:bedrock:*:*:inference-profile/{inference_profile}"
            ]
        },
        {
            "Sid": "AmazonBedrockAgentBedrockGetInferenceProfile",
            "Effect": "Allow",
            "Action":  [
                "bedrock:GetInferenceProfile",
                "bedrock:ListInferenceProfiles",
                "bedrock:UseInferenceProfile"
            ],
            "Resource": [
                f"arn:aws:bedrock:*:*:inference-profile/{inference_profile}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

In [52]:
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Allow",
        "Principal": {
            "Service": "bedrock.amazonaws.com"
        },
        "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)

iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': 'a6e2ea7b-c294-4089-9ea1-00fc03eb0012',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 28 Aug 2025 14:50:29 GMT',
   'x-amzn-requestid': 'a6e2ea7b-c294-4089-9ea1-00fc03eb0012',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [63]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel="mistral.mistral-large-2402-v1:0",
    instruction=agent_instruction,
)
agent_id = response['agent']['agentId']
agent_id, response

('BI9W1TIJTY',
 {'ResponseMetadata': {'RequestId': '7ce5fab0-0282-47ed-ab39-b38d091748f0',
   'HTTPStatusCode': 202,
   'HTTPHeaders': {'date': 'Thu, 28 Aug 2025 15:05:35 GMT',
    'content-type': 'application/json',
    'content-length': '698',
    'connection': 'keep-alive',
    'x-amzn-requestid': '7ce5fab0-0282-47ed-ab39-b38d091748f0',
    'x-amz-apigw-id': 'QBZ54F0YIAMEvJw=',
    'x-amzn-trace-id': 'Root=1-68b0703e-02474a307b82d60852db16f1'},
   'RetryAttempts': 0},
  'agent': {'agentId': 'BI9W1TIJTY',
   'agentName': 'hr-assistant-function-def',
   'agentArn': 'arn:aws:bedrock:us-east-1:637423659006:agent/BI9W1TIJTY',
   'instruction': 'You are an HR agent, helping employees understand HR policies and manage vacation time',
   'agentStatus': 'CREATING',
   'foundationModel': 'mistral.mistral-large-2402-v1:0',
   'description': 'Agent for providing HR assistance to manage vacation time',
   'orchestrationType': 'DEFAULT',
   'idleSessionTTLInSeconds': 1800,
   'agentResourceRoleAr

In [54]:
agent_id

'SRTIYSXNR0'

In [55]:

agent_functions = [
    {
        'name': 'get_available_vacations_days',
        'description': 'get the number of vacations available for a certain employee',
        'parameters': {
            "employee_id": {
                "description": "the id of the employee to get the available vacations",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'reserve_vacation_time',
        'description': 'reserve vacation time for a specific employee - you need all parameters to reserve vacation time',
        'parameters': {
            "employee_id": {
                "description": "the id of the employee for which time off will be reserved",
                "required": True,
                "type": "integer"
            },
            "start_date": {
                "description": "the start date for the vacation time",
                "required": True,
                "type": "string"
            },
            "end_date": {
                "description": "the end date for the vacation time",
                "required": True,
                "type": "string"
            }
        }
    },
]

In [56]:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

agent_action_group_response

{'ResponseMetadata': {'RequestId': 'd03801b4-6740-4801-a1a9-8f66ba05ecfd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 28 Aug 2025 14:51:18 GMT',
   'content-type': 'application/json',
   'content-length': '1335',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd03801b4-6740-4801-a1a9-8f66ba05ecfd',
   'x-amz-apigw-id': 'QBXz9H54IAMEahA=',
   'x-amzn-trace-id': 'Root=1-68b06ce5-2eaa0855007ccfe55b17b983'},
  'RetryAttempts': 0},
 'agentActionGroup': {'agentId': 'SRTIYSXNR0',
  'agentVersion': 'DRAFT',
  'actionGroupId': 'OQPICOJLSA',
  'actionGroupName': 'VacationsActionGroup',
  'description': 'Actions for getting the number of available vacations days for an employee and confirm new time off',
  'createdAt': datetime.datetime(2025, 8, 28, 14, 51, 17, 923768, tzinfo=tzlocal()),
  'updatedAt': datetime.datetime(2025, 8, 28, 14, 51, 17, 923768, tzinfo=tzlocal()),
  'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-east-1:637423659006:function:hr-assistant-function-

In [64]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
response

{'ResponseMetadata': {'RequestId': '4fee0ca4-9f24-486f-adb8-e863a70d40c6',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 28 Aug 2025 15:05:50 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4fee0ca4-9f24-486f-adb8-e863a70d40c6',
   'x-amz-apigw-id': 'QBZ8OEP-IAMEvCA=',
   'x-amzn-trace-id': 'Root=1-68b0704d-1e99097748bb3ff967b2aeb0'},
  'RetryAttempts': 0},
 'agentId': 'BI9W1TIJTY',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2025, 8, 28, 15, 5, 50, 136976, tzinfo=tzlocal())}

In [65]:
response = bedrock_agent_client.create_agent_alias(
    agentAliasName='test-alias-1',
    agentId="BI9W1TIJTY"
)

response

{'ResponseMetadata': {'RequestId': '8e2027eb-b8c2-4e5f-b0a8-35de1d88a0d7',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 28 Aug 2025 15:05:58 GMT',
   'content-type': 'application/json',
   'content-length': '382',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8e2027eb-b8c2-4e5f-b0a8-35de1d88a0d7',
   'x-amz-apigw-id': 'QBZ9lHnHoAMEHAg=',
   'x-amzn-trace-id': 'Root=1-68b07056-52fcbf312126a47230561d0f'},
  'RetryAttempts': 0},
 'agentAlias': {'agentId': 'BI9W1TIJTY',
  'agentAliasId': '2IEI9NIVOA',
  'agentAliasName': 'test-alias-1',
  'agentAliasArn': 'arn:aws:bedrock:us-east-1:637423659006:agent-alias/BI9W1TIJTY/2IEI9NIVOA',
  'routingConfiguration': [{}],
  'createdAt': datetime.datetime(2025, 8, 28, 15, 5, 58, 689905, tzinfo=tzlocal()),
  'updatedAt': datetime.datetime(2025, 8, 28, 15, 5, 58, 689905, tzinfo=tzlocal()),
  'agentAliasStatus': 'CREATING',
  'aliasInvocationState': 'ACCEPT_INVOCATIONS'}}

In [66]:
agent_alias_id = "2IEI9NIVOA"

In [60]:
print(agent_id,agent_alias_id)

SRTIYSXNR0 SNCCSH316L


In [67]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="How much vacation does the employee with employee_id set to 3 have available?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2025-08-28 15:06:09,794] p16253 {2674590716.py:16} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Thu, 28 Aug 2025 15:06:09 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '8798ecd4-8420-11f0-92c6-02eb5d432ec1',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '8f873b5d-49cd-4009-9eef-d4b7924d7186'},
                      'HTTPStatusCode': 200,
                      'RequestId': '8f873b5d-49cd-4009-9eef-d4b7924d7186',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7fabcd862080>,
 'contentType': 'application/json',
 'sessionId': '8798ecd4-8420-11f0-92c6-02eb5d432ec1'}


[2025-08-28 15:06:13,879] p16253 {2674590716.py:23} INFO - Final answer ->
The employee with the ID 3 has 10 days of vacation available.


In [66]:
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="Great. please reserve one day of time off for the employee with employee_id set to 1 for 2025-12-30",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2025-08-25 21:17:56,488] p9121 {505939497.py:10} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Mon, 25 Aug 2025 21:17:56 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': 'f65233c6-81f8-11f0-a836-0e692c5ba471',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '7655fd6f-91ea-452a-bec1-468210679f9f'},
                      'HTTPStatusCode': 200,
                      'RequestId': '7655fd6f-91ea-452a-bec1-468210679f9f',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7fbda0b90580>,
 'contentType': 'application/json',
 'sessionId': 'f65233c6-81f8-11f0-a836-0e692c5ba471'}


Exception: ('unexpected event.', EventStreamError('An error occurred (accessDeniedException) when calling the InvokeAgent operation: Access denied when calling Bedrock. Check your request permissions and retry the request.'))